<a href="https://colab.research.google.com/github/Jushef/Azure-AutoML/blob/main/AutoML%20Jupyter%20Notebook/Forecasting_AutoML.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# Load in Azure Packages
from azureml.core import Workspace, Dataset, Datastore
from azureml.core import Experiment
from azureml.core.compute import ComputeTarget
from azureml.train.automl import AutoMLConfig
from azureml.train.automl.run import AutoMLRun
from azureml.widgets import RunDetails
from azureml.opendatasets import OjSalesSimulated
from azureml.automl.core.forecasting_parameters import ForecastingParameters

In [ ]:
# Load in non-Azure Packages
import pandas as pd
import numpy as np
import os
from pathlib import Path

In [ ]:
# Connect to your AMLS Workspace
ws = Workspace.from_config()

In [ ]:
# Set your Compute Cluster
compute_name = 'automl-compute'
compute_target = ComputeTarget(ws, compute_name)

In [ ]:
# Set your Datastore
datastore = Datastore.get_default(ws)
my_datastore_name = 'workspaceblobstore'
my_datastore = Datastore.get(ws, my_datastore_name)

In [ ]:
# Retrieve the OJ Sales Data Set
oj_sales_files = OjSalesSimulated.get_file_dataset()
oj_sales = oj_sales_files.take(10)

In [ ]:
# Create a folder called OJ_Sales
folder_name = "OJ_Sales"
os.makedirs(folder_name, exist_ok=True)

In [ ]:
# Download the OJ Sales Files to your OJ_Sales folder
oj_sales.download(folder_name, overwrite=True)

['/mnt/batch/tasks/shared/LS_root/mounts/clusters/inv07021-automl/code/OJ_Sales/https%3A/%2Fazureopendatastorage.azurefd.net/ojsales-simulatedcontainer/oj_sales_data/Store1000_dominicks.csv',
 '/mnt/batch/tasks/shared/LS_root/mounts/clusters/inv07021-automl/code/OJ_Sales/https%3A/%2Fazureopendatastorage.azurefd.net/ojsales-simulatedcontainer/oj_sales_data/Store1000_minute.maid.csv',
 '/mnt/batch/tasks/shared/LS_root/mounts/clusters/inv07021-automl/code/OJ_Sales/https%3A/%2Fazureopendatastorage.azurefd.net/ojsales-simulatedcontainer/oj_sales_data/Store1000_tropicana.csv',
 '/mnt/batch/tasks/shared/LS_root/mounts/clusters/inv07021-automl/code/OJ_Sales/https%3A/%2Fazureopendatastorage.azurefd.net/ojsales-simulatedcontainer/oj_sales_data/Store1001_dominicks.csv',
 '/mnt/batch/tasks/shared/LS_root/mounts/clusters/inv07021-automl/code/OJ_Sales/https%3A/%2Fazureopendatastorage.azurefd.net/ojsales-simulatedcontainer/oj_sales_data/Store1001_minute.maid.csv',
 '/mnt/batch/tasks/shared/LS_root/mo

In [ ]:
# Create a single Pandas dataframe out of your OJ Files
OJ_file_path = Path('OJ_Sales').rglob('*.csv')
OJ_files = [x for x in OJ_file_path]
df = pd.concat((pd.read_csv(f) for f in OJ_files))

In [ ]:
# View your Pandas dataframe
df.head(10)

,WeekStarting,Store,Brand,Quantity,Advert,Price,Revenue
0,1990-06-14,1000,dominicks,12003,1,2.59,31087.77
1,1990-06-21,1000,dominicks,10239,1,2.39,24471.21
2,1990-06-28,1000,dominicks,17917,1,2.48,44434.16
3,1990-07-05,1000,dominicks,14218,1,2.33,33127.94
4,1990-07-12,1000,dominicks,15925,1,2.01,32009.25
5,1990-07-19,1000,dominicks,17850,1,2.17,38734.50
6,1990-07-26,1000,dominicks,10576,1,1.97,20834.72
7,1990-08-02,1000,dominicks,9912,1,2.26,22401.12
8,1990-08-09,1000,dominicks,9571,1,2.11,20194.81
9,1990-08-16,1000,dominicks,15748,1,2.42,38110.16


In [ ]:
# Register OJ Sales as a dataset
Dataset.Tabular.register_pandas_dataframe(df, datastore,
                            "OJ Sales Sample")

Validating arguments.
Arguments validated.
Successfully obtained datastore reference and path.
Uploading file to managed-dataset/3419f424-572b-452c-943b-07248823e9e2/
Successfully uploaded file to datastore.
Creating and registering a new dataset.
Successfully created and registered a new dataset.


{
  "source": [
    "('workspaceblobstore', 'managed-dataset/3419f424-572b-452c-943b-07248823e9e2/')"
  ],
  "definition": [
    "GetDatastoreFiles",
    "ReadParquetFile",
    "DropColumns"
  ],
  "registration": {
    "id": "c7f3262f-e3f6-4b23-a21f-7922deb9bd0f",
    "name": "OJ Sales Sample",
    "version": 1,
    "workspace": "Workspace.create(name='automl-example-workspace', subscription_id='0aad4149-65b5-42c8-9b25-ce17f2fed264', resource_group='auto-ml-example-resource-group')"
  }
}

In [ ]:
# Set your experiment that will be used to train your AutoML Model
experiment_name = 'OJ-Sales-Forecasting'
exp = Experiment(workspace=ws, name=experiment_name)

In [ ]:
# Retrieve your OJ Sales Sample dataset
dataset_name = "OJ Sales Sample"
dataset = Dataset.get_by_name(ws, dataset_name, version='latest')

In [ ]:
# Set your target column
target_column = 'Quantity'

In [ ]:
# Set your task to forecasting
task = 'forecasting'

In [ ]:
# Set the metric you will use to score your model
primary_metric = 'normalized_root_mean_squared_error'

In [ ]:
# Set featurization to auto
featurization = 'auto'

In [ ]:
# Set forecasting parameters
params=ForecastingParameters.from_parameters_dict( {'country_or_region_for_holidays':'US',\
                                                    'drop_columns_names':'Revenue',\
                                                    'forecast_horizon': 6,\
                                                    'target_rolling_window_size': 'auto',\
                                                    'target_lags': 'auto',\
                                                    'feature_lags': 'auto',\
                                                    'seasonality': 'auto',\
                                                    'short_series_handling': True,\
                                                    'use_stl': 'season_trend',\
                                                    'time_column_name':'WeekStarting',\
                                                    'time_series_id_column_names':['Store','Brand'],\
                                                    'short_series_handling_configuration': 'auto'},\
                                                    validate_params=True)

In [ ]:
# Set AutoML configuration
config = AutoMLConfig(task=task,
                     primary_metric=primary_metric,
                     featurization=featurization,
                     compute_target=compute_target,
                     training_data=dataset,
                     label_column_name=target_column,
                     experiment_timeout_minutes=15,
                     enable_early_stopping=True,
                     n_cross_validations=3,
                     model_explainability=True,
                     enable_stack_ensemble=False,
                     enable_voting_ensemble=True,
                     forecasting_parameters=params)

In [ ]:
# Train your AutoML model to forecast OJ Sales
AutoML_run = exp.submit(config, show_output = True)
RunDetails(AutoML_run).show()

Submitting remote run.
No run_configuration provided, running on automl-compute with default configuration
Running on remote compute: automl-compute


Experiment,Id,Type,Status,Details Page,Docs Page
OJ-Sales-Forecasting,AutoML_6a2a167f-2ea6-445c-8db7-bb7833cfe72d,automl,NotStarted,Link to Azure Machine Learning studio,Link to Documentation



Current status: DatasetFeaturizationCompleted. Completed featurizing the CV split.
Current status: ModelSelection. Beginning model selection.
Heuristic parameters: Target_Lag = '[0]', Target_Rolling_Window = '0'.


********************************************************************************************
DATA GUARDRAILS: 

TYPE:         Time Series ID detection
STATUS:       PASSED
DESCRIPTION:  The data set was analyzed, and no duplicate time index were detected.
              Learn more about time-series forecasting configurations: https://aka.ms/AutomatedMLForecastingConfiguration

********************************************************************************************

TYPE:         Frequency detection
STATUS:       PASSED
DESCRIPTION:  The time series was analyzed, all data points are aligned with detected frequency.
              

********************************************************************************************

TYPE:         Missing feature values imputation
S

_AutoMLWidget(widget_settings={'childWidgetDisplay': 'popup', 'send_telemetry': False, 'log_level': 'INFO', 's…

In [ ]:
# Set forecasting parameters to train ARIMA and Prophet Models
params2=ForecastingParameters.from_parameters_dict({'country_or_region_for_holidays':None,\
                                                    'drop_columns_names':['Revenue','Price','Advert'],\
                                                    'forecast_horizon': 6,\
                                                    'target_rolling_window_size': None,\
                                                    'target_lags': None,\
                                                    'feature_lags': None,\
                                                    'seasonality': 'auto',\
                                                    'short_series_handling': True,\
                                                    'use_stl': 'season_trend',\
                                                    'time_column_name':'WeekStarting',\
                                                    'time_series_id_column_names':['Store','Brand'],
                                                    'short_series_handling_configuration': 'auto'},\
                                                    validate_params=True)

In [ ]:
# Set AutoML configuration passing in new forecasting parameters
config2=AutoMLConfig(task=task,
                     primary_metric=primary_metric,
                     featurization=featurization,
                     compute_target=compute_target,
                     training_data=dataset,
                     label_column_name=target_column,
                     experiment_timeout_minutes=15,
                     enable_early_stopping=True,
                     n_cross_validations=3,
                     model_explainability=True,
                     enable_stack_ensemble=False,
                     enable_voting_ensemble=True,
                     forecasting_parameters=params2)

In [ ]:
# Train your AutoML model to forecast OJ Sales using Prophet and ARIMA
AutoML_Prophet_ARIMA_run = exp.submit(config2, show_output = True)
RunDetails(AutoML_Prophet_ARIMA_run).show()

Submitting remote run.
No run_configuration provided, running on automl-compute with default configuration
Running on remote compute: automl-compute


Experiment,Id,Type,Status,Details Page,Docs Page
OJ-Sales-Forecasting,AutoML_acf40396-ac33-482f-a70c-ccdd015e4ccc,automl,NotStarted,Link to Azure Machine Learning studio,Link to Documentation



Current status: DatasetFeaturization. Beginning to featurize the CV split.
Current status: ModelSelection. Beginning model selection.

********************************************************************************************
DATA GUARDRAILS: 

TYPE:         Time Series ID detection
STATUS:       PASSED
DESCRIPTION:  The data set was analyzed, and no duplicate time index were detected.
              Learn more about time-series forecasting configurations: https://aka.ms/AutomatedMLForecastingConfiguration

********************************************************************************************

TYPE:         Frequency detection
STATUS:       PASSED
DESCRIPTION:  The time series was analyzed, all data points are aligned with detected frequency.
              

********************************************************************************************

TYPE:         Missing feature values imputation
STATUS:       PASSED
DESCRIPTION:  No feature missing values were detected in the

_AutoMLWidget(widget_settings={'childWidgetDisplay': 'popup', 'send_telemetry': False, 'log_level': 'INFO', 's…

In [ ]:
# Set model description, tags, and give it a name
description = 'Best AutoML Forecasting Run using OJ Sales Sample Data.' 
tags = {'project' : "OJ Sales", "creator" : "your name"} 
model_name = 'OJ-Sales-Sample-Forecasting-AutoML' 

In [ ]:
# Register your model
AutoML_run.register_model(model_name=model_name, description=description, tags=tags)

Model(workspace=Workspace.create(name='automl-example-workspace', subscription_id='0aad4149-65b5-42c8-9b25-ce17f2fed264', resource_group='auto-ml-example-resource-group'), name=OJ-Sales-Sample-Forecasting-AutoML, id=OJ-Sales-Sample-Forecasting-AutoML:1, version=1, tags={'project': 'OJ Sales', 'creator': 'your name'}, properties={})

In [ ]:
# Reregister your model using a different metric
description = 'Best AutoML Forecasting Run using OJ Sales Sample Data.'
tags = {'project' : "OJ Sales", "creator" : "your name", "metric" : "R2 Score"} 
model_name = 'OJ-Sales-Sample-Forecasting-AutoML-R2'
AutoML_run.register_model(model_name=model_name, description=description, tags=tags, metric = 'r2_score')

Model(workspace=Workspace.create(name='automl-example-workspace', subscription_id='0aad4149-65b5-42c8-9b25-ce17f2fed264', resource_group='auto-ml-example-resource-group'), name=OJ-Sales-Sample-Forecasting-AutoML-R2, id=OJ-Sales-Sample-Forecasting-AutoML-R2:1, version=1, tags={'project': 'OJ Sales', 'creator': 'your name', 'metric': 'R2 Score'}, properties={})

In [ ]:
# Retrieve and register your second training run
experiment_name = 'OJ-Sales-Forecasting'
exp = Experiment(workspace=ws, name=experiment_name) 
AutoML_run = AutoMLRun(experiment = exp, run_id = 'AutoML_acf40396-ac33-482f-a70c-ccdd015e4ccc')  
description = 'AutoML Forecasting Run for OJ Sales Data with Prophet/ARIMA.'
tags = {'project' : "OJ Sales", "creator" : "your name"} 
model_name = 'OJ-Sales-Sample-Forecasting-AutoML-ARIMA'
AutoML_run.register_model(model_name=model_name, description=description, tags=tags)

Model(workspace=Workspace.create(name='automl-example-workspace', subscription_id='0aad4149-65b5-42c8-9b25-ce17f2fed264', resource_group='auto-ml-example-resource-group'), name=OJ-Sales-Sample-Forecasting-AutoML-ARIMA, id=OJ-Sales-Sample-Forecasting-AutoML-ARIMA:1, version=1, tags={'project': 'OJ Sales', 'creator': 'your name'}, properties={})